# So sieht die durchschnittliche Bachelor-Kandidatin aus

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import math
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import numpy as np
import re


import time
import datetime


In [57]:
import shutil

## Linkliste für Kandidatinnenseiten zusammensuchen

In [85]:
linklist = []
staffeln = [1,2,3,4,5,6,7,""]
for staffel in staffeln:
    #staffel = 7
    website = requests.get("http://www.3plus.tv/bachelor/"+str(staffel)).text
    soup = BeautifulSoup(website, "html.parser")
    kandidatinnen = soup.find('div', class_='sidebar').find('ul', class_='menu').find_all('li', class_="leaf")[-23:]

    if staffel == "": # weil die URL für die aktuelle Staffel auf "" lautet...
        staffel = 8
    for k in kandidatinnen:
        linklist.append([staffel, k.text, k.find('a')['href']])

In [86]:
linklist

[[1, 'Episode 2', '/episode/der-bachelor/staffel-1-folge-2'],
 [1, 'Episode 1', '/episode/der-bachelor/staffel-1-folge-1'],
 [1, 'Sponsor', '/bachelor/sponsor1'],
 [1, 'Cam', '/bachelor/1/cam'],
 [1, 'Daryna', '/bachelor/1/daryna'],
 [1, 'Delilah und Raphaela', '/bachelor/1/delilahraphaela'],
 [1, 'Désirée', '/bachelor/1/Desiree'],
 [1, 'Estelle', '/bachelor/1/estelle'],
 [1, 'JLo', '/bachelor/1/jlo'],
 [1, 'Larissa', '/bachelor/1/larissa'],
 [1, 'Linda', '/bachelor/1/linda'],
 [1, 'Michèle', '/bachelor/1/michele'],
 [1, 'Nathalie', '/bachelor/1/nathalie'],
 [1, 'Nayla', '/bachelor/1/nayla'],
 [1, 'Rachele', '/bachelor/1/rachele'],
 [1, 'Sabrina', '/bachelor/1/sabrina'],
 [1, 'Sandy', '/bachelor/1/sandy'],
 [1, 'Senait', '/bachelor/1/senait'],
 [1, 'Sunny', '/bachelor/1/sunny'],
 [1, 'Tamara', '/bachelor/1/tamara'],
 [1, 'Tschosy', '/bachelor/1/tschosy'],
 [1, 'Vanessa', '/bachelor/1/vanessa'],
 [1, 'Viktoria', '/bachelor/1/viktoria'],
 [2, 'Episode 2', '/episode/der-bachelor/staffe

## Bilder abrufen 

In [87]:
dataset = []
for kandidatin in linklist:
    # Ausnahme-Seiten wie "Highlights" und Co entfernen
    if ("Episode" in kandidatin[1]) or ("Sponsor" in kandidatin[1]) or ("Damen" in kandidatin[1]) or ("Highlights" in kandidatin[1]):
        continue
        
    website = requests.get("http://www.3plus.tv"+kandidatin[2]).text
    soup = BeautifulSoup(website, "html.parser")
    
    try:
        desc = soup.find('div', {'id':'content-content-content'})
        img = desc.find('img')['src']
        about = desc.find_all('p')[1].text
        alter = desc.find_all('table')[1].find_all('td')[1].text
        wohnort = desc.find_all('table')[1].find_all('td')[3].text
        sternzeichen = desc.find_all('table')[1].find_all('td')[5].text
        dataset.append([kandidatin[0], kandidatin[1], kandidatin[2], img, about, alter, wohnort, sternzeichen])
    except:
        try:
            img = desc.find('img')['src']
            about = desc.find_all('p')[0].text
            alter = desc.find_all('table')[0].find_all('td')[1].text
            wohnort = desc.find_all('table')[0].find_all('td')[3].text
            sternzeichen = desc.find_all('table')[0].find_all('td')[5].text
            dataset.append([kandidatin[0], kandidatin[1], kandidatin[2], img, about, alter, wohnort, sternzeichen])
        except:
            print("Problem mit Beschreibung: "+kandidatin[1]+", Staffel "+str(kandidatin[0]))
     
    try:
        response = requests.get("http://www.3plus.tv"+img, stream=True)

        local_file = open('Bilder/'+str(kandidatin[0])+"_"+kandidatin[1]+'.jpg', 'wb')
        response.raw.decode_content = True
        shutil.copyfileobj(response.raw, local_file)
        del response
    except:
        print("Problem mit Foto: "+kandidatin[1]+", Staffel "+str(kandidatin[0]))

Problem mit Beschreibung: Die Damen, Staffel 3
Problem mit Beschreibung: Sofia, Staffel 3
Problem mit Beschreibung: Highlights, Staffel 4
Problem mit Foto: Highlights, Staffel 4
Problem mit Beschreibung: Die Damen, Staffel 4
Problem mit Beschreibung: Highlights, Staffel 5
Problem mit Foto: Highlights, Staffel 5


## Daten analysieren

In [88]:
df = pd.DataFrame(dataset, columns=["Staffel", "Name", "Link", "Foto", "Beschreibung", "Alter", "Kanton", "Sternzeichen"])

In [89]:
df['Alter'] = pd.to_numeric(df['Alter'])

In [90]:
df.mean()

Staffel     4.590361
Alter      25.781818
dtype: float64

In [91]:
df['Kanton'].value_counts()

Kanton Zürich                    49
Kanton Bern                      17
Kanton Aargau                    15
Kanton Solothurn                 11
Kanton Luzern                    11
Kanton Thurgau                   10
Kanton St. Gallen                 9
Deutschland                       7
Kanton Schwyz                     4
Kanton Basel-Stadt                2
Köln Deutschland                  2
Zürich                            2
Kanton Freiburg                   2
Österreich                        2
Liechtenstein                     2
Berlin                            2
Kanton Appenzell Ausserrhoden     2
                                  1
Kanton Tessin                     1
Kanton Basel Land                 1
Mailand                           1
Kanton Basel-Landschaft           1
Kanton Basel Stadt                1
Kanton Basel                      1
Kanton Schaffhausen               1
Italien                           1
Kanton Zug                        1
Kanton St.Gallen            

In [92]:
df['Sternzeichen'].value_counts()

Krebs         27
Steinbock     19
Zwillinge     18
Löwe          14
Skorpion      14
Wassermann    13
Fische        12
Schütze       12
Stier         11
Waage          8
Widder         6
Jungfrau       6
Zwilling       3
Fisch          2
               1
Name: Sternzeichen, dtype: int64

## Average face berechnen
Nach Anleitung von stekhn
https://github.com/stekhn/average-faces-opencv

In [ ]:
!python extract.py shape_predictor_68_face_landmarks.dat ./images

In [ ]:
!python average.py ./images 170 240